In [1]:
import tensorflow as tf
import numpy as np

In [2]:
n_inputs = 784
n_hidden1 = 512
n_hidden2 = 256
n_outputs = 10

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/home/anand/.tensorflow/data/MNIST")

Extracting /home/anand/.tensorflow/data/MNIST/train-images-idx3-ubyte.gz
Extracting /home/anand/.tensorflow/data/MNIST/train-labels-idx1-ubyte.gz
Extracting /home/anand/.tensorflow/data/MNIST/t10k-images-idx3-ubyte.gz
Extracting /home/anand/.tensorflow/data/MNIST/t10k-labels-idx1-ubyte.gz


In [4]:
x = tf.placeholder(dtype=tf.float32, shape=(None, n_inputs))
y = tf.placeholder(dtype=tf.int64, shape=(None))

In [6]:
def new_layer(x, n_nodes, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(x.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal(shape=(n_inputs, n_nodes), stddev=stddev)
        w = tf.Variable(init)
        b = tf.Variable(tf.zeros(shape=(n_nodes)))
        z = tf.matmul(x, w) + b
        if activation == 'relu':
            return tf.nn.relu(z)
        return z

In [7]:
with tf.name_scope('dnn'):
    hidden1 = new_layer(x, n_hidden1, 'hidden1', activation='relu')
    hidden2 = new_layer(hidden1, n_hidden2, 'hidden2', activation='relu')
    output = new_layer(hidden2, n_outputs, 'outputs')

In [9]:
with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=output)
    loss = tf.reduce_mean(xentropy)

In [17]:
learning_rate = 0.1
n_epocs = 100
batch_size = 100

In [12]:
with tf.name_scope('optimizer'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)

In [16]:
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(output, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [19]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [22]:
with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epocs):
        for batch in range(mnist.train.num_examples // batch_size):
            x_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={x: x_batch, y: y_batch})
        print(loss.eval(feed_dict={x: x_batch, y: y_batch}))
            
    saver.save(sess, '/tmp/model/mnist2.ckpt')

0.154343
0.0401526
0.0443916
0.0804484
0.0306493
0.0129843
0.0259539
0.0109027
0.0138669
0.00661842
0.00855853
0.00605331
0.0041816
0.00945966
0.00403831
0.00237641
0.00256068
0.00571599
0.00340799
0.00330485
0.00311339
0.00318168
0.00682412
0.00164893
0.0022426
0.00408405
0.00325012
0.00120595
0.00120061
0.00113647
0.00128533
0.00124588
0.000886322
0.00134719
0.00131497
0.00155856
0.00103687
0.00106031
0.00068626
0.000662867
0.00137997
0.00120653
0.00134627
0.000721679
0.000506059
0.00117539
0.00121172
0.000239375
0.000691923
0.000747952
0.000981891
0.000466485
0.000771421
0.000671272
6.97411e-05
0.000343586
0.000484366
0.000646863
0.000637108
0.00106618
0.000835594
0.000249171
0.000886304
0.000551715
0.000945963
0.000257016
0.000209132
0.000115864
0.000219211
0.000318958
0.000363905
0.00012012
0.000246921
0.000373738
0.000367951
0.000409053
0.000508425
0.000161119
0.000384895
0.000440113
0.000553836
0.000463745
0.000493104
0.000319771
0.000208898
0.00013898
0.000551556
0.000180507
0.

In [36]:
idx = 131

with tf.Session() as sess:
    save_path = saver.restore(sess, '/tmp/model/mnist2.ckpt')
    
    image = mnist.test.images[idx].reshape(-1, 784)
    l = sess.run(output, feed_dict={x: image})
    print("Prediction", np.argmax(l, axis=1))
    print("Actual", mnist.test.labels[idx])

INFO:tensorflow:Restoring parameters from /tmp/model/mnist2.ckpt
Prediction [6]
Actual 6
